In [1]:
import pandas as pd
import numpy as np
import pathlib
from tqdm.notebook import tqdm
import librosa
import awkward as ak
import matplotlib.pyplot as plt

In [2]:
dataset_folder = pathlib.Path("RavdessAudioOnly")  # change it with your file location

In [3]:
file_list = list(dataset_folder.iterdir())
for i, path in enumerate(file_list):
    if path.suffix != ".wav":
        file_list.pop(i)
file_list = sorted(file_list)

In [4]:
b = ak.ArrayBuilder()
for path in tqdm(file_list):
    librosa_audio_segment, sr = librosa.load(path, sr=22042)
    trimmed_audio, idx = librosa.effects.trim(librosa_audio_segment) 
    b.begin_list()
    b.begin_list()
    for value in trimmed_audio:
        b.real(value)
    b.end_list() 
    b.end_list() 
ts_array = b.snapshot()

  0%|          | 0/2462 [00:00<?, ?it/s]

In [5]:
#MOLTO LENTO
ts = ak.to_dataframe(ts_array)
ts = ts.reset_index([0,1])

In [6]:
df = pd.DataFrame()
for i in range(2452):
    df = pd.concat([df, ts[ts['entry'] == i]['values']], axis = 1)


In [8]:
file_names = list()
for path in file_list:
    stem = path.stem.split("-")
    file_names.append(path.name)


In [18]:
df

,values,values,values,values,values,values,values,values,values,values,...,values,values,values,values,values,values,values,values,values,values
0,0.000009,-0.000002,2.245103e-07,3.099649e-07,8.127711e-08,0.0,-0.000008,-0.000103,0.000606,-0.000008,...,-2.044693e-06,0.000009,-0.000083,4.075184e-06,-0.000019,0.000042,-0.000020,2.442115e-06,-0.000075,0.000037
1,0.000006,-0.000002,-2.474420e-07,-3.249839e-07,-5.978953e-08,0.0,-0.000025,-0.000173,0.000932,0.000014,...,-9.399268e-07,0.000025,-0.000113,1.219346e-05,-0.000015,0.000053,-0.000311,7.042823e-07,-0.000098,-0.000039
2,0.000010,0.000003,2.745297e-07,3.398443e-07,-4.857318e-07,0.0,0.000058,-0.000142,0.000828,0.000011,...,-3.845220e-05,0.000009,-0.000073,-3.600323e-07,-0.000009,0.000057,-0.000050,-1.044781e-05,-0.000029,-0.000056
3,-0.000003,-0.000004,-3.065930e-07,-3.542281e-07,1.014322e-05,0.0,-0.000003,-0.000161,0.000895,-0.000004,...,-3.264781e-06,0.000001,-0.000141,1.032179e-05,-0.000020,0.000042,0.000414,-6.222557e-06,-0.000080,0.000034
4,0.000002,-0.000011,3.444849e-07,3.677032e-07,1.876014e-05,0.0,0.000030,-0.000131,0.000836,-0.000010,...,-8.016507e-06,-0.000001,-0.000235,6.437055e-06,-0.000011,0.000032,-0.000363,2.210542e-06,-0.000085,0.000046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126459,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#CATEGORICAL DF
CATEGORICAL_FEATURES_NAMES = {
    "modality": {"01": "full-AV", "02": "video-only", "03": "audio-only"}, 
    "vocal_channel": {"01": "speech", "02": "song"},
    "emotion": {"01" : "neutral", "02" : "calm", "03" : "happy", "04" : "sad", "05" : "angry", "06" : "fearful", "07" : "disgust", "08" : "surprised"},
    "emotional_intensity": {"01" : "normal", "02" : "strong"},
    "statement": {"01" : "Kids are talking by the door", "02" : "Viva il che"},
    "repetition": {"01" : "1st", "02" : "2nd"},
    "actor": {str(i).zfill(2): str(i).zfill(2) for i in range(1, 25)}
}

In [15]:
cat = list()
file_names = list()
for path in file_list:
    stem = path.stem.split("-")
    file_names.append(path.name)
    cat.append(stem)
cat = pd.DataFrame(cat)
cat.columns = list(CATEGORICAL_FEATURES_NAMES.keys())


In [16]:
for column in cat.columns:
    cat = cat.replace(CATEGORICAL_FEATURES_NAMES)

In [17]:
cat["sex"] = ["F" if i % 2 == 0 else "M" for i in cat["actor"].astype(int)]
cat["filename"] = file_names


ValueError: invalid literal for int() with base 10: '19(1)'

In [ ]:
#MERGE SU UNICO DATAFRAME
final_df = pd.merge(df,cat, how = 'inner', on = 'filename')

In [ ]:
final_df.to_csv('ts.csv', index= False)